In [ ]:
import numpy as np
import pandas as pd
import sys
import os
import simtk.openmm as mm
import simtk.openmm.app as app
import simtk.unit as unit
import mdtraj
import warnings
warnings.filterwarnings('ignore')

sys.path.append('../..')

from OpenSMOG3SPN2.forcefields.parsers import SMOGParser, DNA3SPN2Parser
from OpenSMOG3SPN2.forcefields import SMOG3SPN2Model
from OpenSMOG3SPN2.utils.helper_functions import get_WC_paired_seq
from OpenSMOG3SPN2.utils.chromatin_helper_functions import get_chromatin_rigid_bodies
from OpenSMOG3SPN2.utils.insert import insert_molecules
from OpenSMOG3SPN2.forcefields.rigid import createRigidBodies


We build a simulation box fulfilled with nucleosomes. For simplicity, in this tutorial, we only put 5 nucleosomes in the box.

In [ ]:
n_nucl = 5
box_a, box_b, box_c = 30, 30, 30
multi_nucl = SMOG3SPN2Model()


In [ ]:
# load single nucleosomes, first load histone, then load DNA
histone_parser = SMOGParser.from_atomistic_pdb('single-nucl-pdb-files/histone.pdb', 'cg_single_nucl_histone.pdb', 
                                               default_parse=False)
histone_parser.parse_mol(get_native_pairs=False)
with open('single_nucl_dna_seq.txt', 'r') as f:
    seq1 = f.readlines()[0].strip()
seq2 = get_WC_paired_seq(seq1)
target_seq = seq1 + seq2
dna_parser = DNA3SPN2Parser.from_atomistic_pdb('single-nucl-pdb-files/dna.pdb', 'cg_single_nucl_dna.pdb', 
                                               new_sequence=target_seq, temp_name='dna')
single_nucl = SMOG3SPN2Model()
single_nucl.append_mol(histone_parser)
single_nucl.append_mol(dna_parser)
single_nucl.atoms_to_pdb('cg_single_nucl.pdb')

In [ ]:
# prepare pdb
if not os.path.exists('start.pdb'):
    insert_molecules('cg_single_nucl.pdb', 'start.pdb', n_mol=n_nucl, 
                     box=[box_a, box_b, box_c])
top = app.PDBFile('start.pdb').getTopology()
init_coord = app.PDBFile('start.pdb').getPositions()
rigid_coord = init_coord

for i in range(n_nucl):
    multi_nucl.append_mol(histone_parser)
    multi_nucl.append_mol(dna_parser)

In [ ]:
# set rigid bodies
rigid_bodies = []
single_nucl_rigid_body = np.array(get_chromatin_rigid_bodies(n_nucl=1, nrl=147)[0])
n_atoms_per_nucl = len(single_nucl.atoms.index)
for i in range(n_nucl):
    rigid_bodies.append((single_nucl_rigid_body + i*n_atoms_per_nucl).tolist())

In [ ]:
# set system and add forces
# no need to add dihedrals and native pairs as we use rigid bodies
multi_nucl.create_system(top, box_a=box_a, box_b=box_b, box_c=box_c)
createRigidBodies(multi_nucl.system, rigid_coord, rigid_bodies)
multi_nucl.add_protein_bonds(force_group=1)
multi_nucl.add_protein_angles(force_group=2)
multi_nucl.add_dna_bonds(force_group=5)
multi_nucl.add_dna_angles(force_group=6)
multi_nucl.add_dna_stackings(force_group=7)
multi_nucl.add_dna_dihedrals(force_group=8)
multi_nucl.add_dna_base_pairs(force_group=9)
multi_nucl.add_dna_cross_stackings(force_group=10)
multi_nucl.parse_all_exclusions()
multi_nucl.add_all_vdwl(force_group=11)
multi_nucl.add_all_elec(force_group=12)

In [ ]:
temperature = 300*unit.kelvin
friction_coeff = 0.01/unit.picosecond
timestep = 10*unit.femtosecond
integrator = mm.LangevinMiddleIntegrator(temperature, friction_coeff, timestep)
multi_nucl.set_simulation(integrator, platform_name='CPU', init_coord=init_coord)
multi_nucl.simulation.minimizeEnergy()
multi_nucl.add_reporters(report_interval=100, output_dcd='output.dcd')
multi_nucl.simulation.context.setVelocitiesToTemperature(temperature)
multi_nucl.simulation.step(500)